In [2]:
import gzip
import numpy
import scipy.optimize
import matplotlib.pyplot as plt
from collections import defaultdict

def readGz(f):
  for l in gzip.open(f):
    yield eval(l)
### Rating baseline: compute averages for each user, or return the global average if we've never seen the user before
allRatings = []
userRatings = defaultdict(list)
data = list(readGz("Assignment1/HW3/assignment1/train.json.gz"))

In [37]:
### Initialization ###
features = []
features_price = []
features_time = []
helpful_rate = []
user_helpful = {}
item_helpful = {}
user_outOf = {}
item_outOf = {}
global_sum = 0
global_size = 0


### training dataset ###
# collect number of helpful for a user/item from its previous reviews
for l in data:#[:100000]:
    if l['helpful']['outOf'] != 0:
        user_helpful[l['reviewerID']] = 0
        item_helpful[l['itemID']] = 0
        user_outOf[l['reviewerID']] = 0
        item_outOf[l['itemID']] = 0
        
for l in data:#[:100000]:
    if l['helpful']['outOf'] != 0:
        user_helpful[l['reviewerID']] += l['helpful']['nHelpful']
        item_helpful[l['itemID']] += l['helpful']['nHelpful']
        user_outOf[l['reviewerID']] += l['helpful']['outOf']
        item_outOf[l['itemID']] += l['helpful']['outOf']
        global_sum += l['helpful']['nHelpful']
        global_size += l['helpful']['outOf']
        
        
global_avg = global_sum * 1.0 / global_size

helpful_rate = []
for l in data:#[:100000]:
    if l['helpful']['outOf'] != 0 and 'price' in l and l['price'] >= 100:
        helpful_rate.append(l['helpful']['nHelpful'])# * 1.0 / l['helpful']['outOf'])
        features_price.append([1, l['helpful']['outOf'], l['rating'], l['price']])

theta_price,residuals,rank,s = numpy.linalg.lstsq(features_price, helpful_rate)

helpful_rate = []
for l in data:#[:100000]:
    if l['helpful']['outOf'] >= 0 and l['unixReviewTime'] <= 1.35e9:
        helpful_rate.append(l['helpful']['nHelpful'])# * 1.0 / l['helpful']['outOf'])
        features_time.append([1, l['helpful']['outOf'], l['rating'], l['unixReviewTime']])

theta_time,residuals,rank,s = numpy.linalg.lstsq(features_time, helpful_rate)

helpful_rate = []
for l in data:#[:100000]:
    if l['helpful']['outOf'] != 0:
        helpful_rate.append(l['helpful']['nHelpful'])# * 1.0 / l['helpful']['outOf'])
        words_num = len(l['summary'].split()) #split the words
        if 'price' in l:
            features.append([1, l['helpful']['outOf'], l['rating'], l['price']])
        else:
            features.append([1, l['helpful']['outOf'], l['rating'], 0])

theta,residuals,rank,s = numpy.linalg.lstsq(features, helpful_rate)

In [39]:
### Testing dataset ###
userItemHelpful = {}
helpfulBias = {}

for l in readGz("Assignment1/HW3/assignment1/test_Helpful.json.gz"):
    user, item = l['reviewerID'],l['itemID']
    if l['helpful']['outOf'] != 0:
        words_num = len(l['reviewText'].split()) #split the words
        if l['reviewerID'] in user_helpful:
            a = user_helpful[l['reviewerID']]
            b = user_outOf[l['reviewerID']]
        else:
            a = 0
            b = 0
        if l['itemID'] in item_helpful:
            c = item_helpful[l['itemID']]
            d = item_outOf[l['itemID']]
        else:
            c = 0
            d = 0
        if b + d >= 80 and l['helpful']['outOf'] >= 40:
            percentage = (a + c) * 1.0 / (b + d)
            bias = (percentage - global_avg) * l['helpful']['outOf'] * -0.4
        elif b + d <= 20 and b + d > 0:
            percentage = (a + c) * 1.0 / (b + d)
            bias = (percentage - global_avg) * l['helpful']['outOf'] * 0.3
        else:
            bias = 0
            
        if 'price' in l and l['price'] >= 100:
            if 'price' in l:
               feat_row = [1, l['helpful']['outOf'], l['rating'], l['price']]
            feat = [theta_price[i]*feat_row[i] for i in range(0,len(feat_row))]
        elif l['helpful']['outOf'] >= 5 and l['unixReviewTime'] <= 1.35e9:
            feat_row = [1, l['helpful']['outOf'], l['rating'], l['unixReviewTime']]
            feat = [theta_time[i]*feat_row[i] for i in range(0,len(feat_row))]
        else:
            if 'price' in l:
                feat_row = [1, l['helpful']['outOf'], l['rating'], l['price']]
            else:
                feat_row = [1, l['helpful']['outOf'], l['rating'], 0]
            feat = [theta[i]*feat_row[i] for i in range(0,len(feat_row))]
            
        userItemHelpful[user+item] = sum(feat)
        helpfulBias[user+item] = bias
    else:
        userItemHelpful[user+item] = 0
        helpfulBias[user+item] = 0

In [40]:
### Write to "prediction_Helpful.txt" file ###
predictions = open("Assignment1/HW3/assignment1/predictions_Helpful.txt", 'w')
for l in open("Assignment1/HW3/assignment1/pairs_Helpful.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    u, i, outOf = l.strip().split('-')
    outOf = int(outOf)
    if outOf == 1:
        predictions.write(u + '-' + i + '-' + str(outOf) + ',' + str(1) + '\n')
    elif round(userItemHelpful[u+i] + helpfulBias[u+i]) > outOf:
        predictions.write(u + '-' + i + '-' + str(outOf) + ',' + str(outOf) + '\n')
    elif round(userItemHelpful[u+i] + helpfulBias[u+i]) < 0:
        predictions.write(u + '-' + i + '-' + str(outOf) + ',' + str(round(userItemHelpful[u+i])) + '\n')
    else:
        predictions.write(u + '-' + i + '-' + str(outOf) + ',' + str(int(round(userItemHelpful[u+i] + helpfulBias[u+i]))) + '\n')
            
predictions.close()